# My First Intraday Trading Strategy

This notebook is part of my learning journey.  
I want to practice using **Git, GitHub, and Python** while also testing my **first intraday trading strategy**.  

The strategy I chose is called an **Opening Range Breakout (ORB)**.  
The idea: look at the first few minutes after the market opens, define a price range, and trade depending on whether the price breaks above or below that range.

In [1]:
import yfinance as yf
import datetime as dt
from backtesting import Strategy, Backtest

/Users/melkerliljegren/Documents/Python Repositories/Projects/Intraday_Strategy/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Step 1: Collect Intraday Data

To test the strategy, I will download **2-minute intraday stock data** from Yahoo Finance using the `yfinance` library.  

I will use Apple (AAPL) as my test case, but later I could try other tickers too.

In [6]:
df = yf.download(
    tickers="AAPL",
    interval="2m",
    period='60d'
)

/var/folders/r6/jv1hv35d4rg8crmf3xb99ykc0000gn/T/ipykernel_11539/2803733564.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed


## Step 2: Preprocess the Data

`yfinance` gives the data with a MultiIndex (e.g. first level = `Price, Close, High...`, second level = ticker name).  
For backtesting, I only need flat column names.  

I drop the `"Ticker"` level so the DataFrame has simple columns: `Open, High, Low, Close, Volume`.

In [7]:
df = df.droplevel("Ticker", axis=1)
df

Price,Close,High,Low,Open,Volume
Datetime,,,,,
2025-08-08 13:30:00+00:00,220.259995,221.009995,220.009995,220.820007,2015878
2025-08-08 13:32:00+00:00,219.339996,220.259995,219.250000,220.235001,609933
2025-08-08 13:34:00+00:00,219.956497,219.975006,219.300003,219.339996,637848
2025-08-08 13:36:00+00:00,219.684998,219.970001,219.509995,219.955002,419440
2025-08-08 13:38:00+00:00,219.794495,219.800003,219.389999,219.699707,410645
...,...,...,...,...,...
2025-09-19 19:50:00+00:00,245.360001,246.300003,244.830002,244.960007,2171071
2025-09-19 19:52:00+00:00,245.449997,245.795502,245.190002,245.360001,1021483
2025-09-19 19:54:00+00:00,245.500000,245.800003,244.740005,245.440002,1536959


## Step 3: Define the Opening Range Breakout Strategy

The rules I will use for my **first intraday strategy**:

1. Take the first 5 minutes after the market opens (the "opening range").
2. Record the highest and lowest price during this range.
3. If the price is above the day's open after the range ends → go **long**.
4. If the price is below the day's open after the range ends → go **short**.
5. Use risk management:
   - Risk per trade = 1% of equity
   - Stop loss = the opposite side of the opening range
   - Take profit = 10 × the risk distance
6. Close all positions before market close.

This is a **simple starting point** - the main goal is to learn how to build and backtest an intraday strategy.

In [16]:
class OpeningRangeBreakout(Strategy):
    # Strategy parameters
    open_range_minutes = 4
    last_minute_bar_in_opening_range = dt.time(13, 30 + open_range_minutes)
    exit_minute_bar = dt.time(19, 58)
    risk_percent = 0.01  # 1% of equity
    take_profit_multiplier = 10  # Take profit at 10x the risk
    max_leverage = 4  # 4x leverage
    
    # what we want to initialize in the berginning
    def init(self):
        self.current_day        = None # tracks current day
        self.current_day_open   = None
        self.opening_range_high = None # tracks high of opening range
        self.opening_range_low  = None # tracks low of opening range
        
    # every day is going to have a different opening range high and low
    def _reset_range(self, day, open):
        self.current_day        = day
        self.current_day_open   = open
        self.opening_range_high = None
        self.opening_range_low  = None
        
    def _get_position_size(self, entry_price: float, stop_price: float) -> int:
        per_share_risk = abs(entry_price - stop_price) 

        if per_share_risk == 0:
            return 0

        # Risk-based cap: position that loses 1 % of equity at the stop
        shares_by_risk = (self.risk_percent * self.equity) / per_share_risk

        # Leverage-based cap: shares affordable with 4× buying power
        shares_by_leverage = (self.max_leverage * self.equity) / entry_price

        # Final size: smaller of the two, floored to an int
        return int(min(shares_by_risk, shares_by_leverage))
        
    # function that is called on every new bar
    def next(self):
        # get current time and date
        t = self.data.index[-1]
        current_bar_date = t.date()

        # if new day, reset opening range
        if self.current_day != current_bar_date:
            self._reset_range(current_bar_date, self.data.Open[-1])
            print(t.time())
        
        # if we are still in the opening range period, update high and low
        if t.time() <= self.last_minute_bar_in_opening_range:
            if self.opening_range_high is None:
                self.opening_range_high = self.data.High[-1]
                self.opening_range_low  = self.data.Low[-1]
            else:
                self.opening_range_high = max(self.opening_range_high, self.data.High[-1])
                self.opening_range_low  = min(self.opening_range_low,  self.data.Low[-1])
                
        # right when the opening range closes, decide to go long or short
        if t.time() == self.last_minute_bar_in_opening_range:
          print(f"opening range high is {self.opening_range_high}")
          print(f"opening range low is {self.opening_range_low}")
        
          # only take a position if we are not already in one
          if not self.position:
            # Calculate range size and planned entry price
            range_size = self.opening_range_high - self.opening_range_low
            planned_entry_price = self.data.Close[-1]  # Using current close as entry price
        
            if self.data.Close[-1] > self.current_day_open:
              # Calculate position size based on risk management
              stop_loss_price = self.opening_range_low
              position_size = self._get_position_size(planned_entry_price, stop_loss_price)
              take_profit_price = planned_entry_price + (self.take_profit_multiplier * range_size)
              
              print(f"going long, position size {position_size} at planned price {planned_entry_price}, stop loss {stop_loss_price}")
              self.buy(size=position_size, sl=stop_loss_price, tp=take_profit_price)
              
            elif self.data.Close[-1] < self.current_day_open:
              # Calculate position size based on risk management
              stop_loss_price = self.opening_range_high
              position_size = self._get_position_size(planned_entry_price, stop_loss_price)
              take_profit_price = planned_entry_price - (self.take_profit_multiplier * range_size)
              
              print(f"going short, position size {position_size} shares at planned price {planned_entry_price}, stop loss {stop_loss_price}")
              self.sell(size=position_size, sl=stop_loss_price, tp=take_profit_price)
              
            else:
              print("doing nothing")

        if self.position and t.time() == self.exit_minute_bar:
          print("closing out position")
          self.position.close()        

In [17]:
def per_share_commission(size, price):
    return abs(size) * 0.0005

## Step 4: Backtest the Strategy

I will run the backtest using the `backtesting.py` library.  

Since this is my **first intraday strategy**, the results are less important than the process. The goal is to **learn how everything fits together**.

In [18]:
bt = Backtest(df, OpeningRangeBreakout, cash=25000, 
              commission=per_share_commission, margin=0.25)

stats = bt.run()

from bokeh.io import output_notebook
output_notebook()
bt.plot()
print(stats)

13:32:00
opening range high is 220.25999450683594
opening range low is 219.25
going short, position size 454 shares at planned price 219.9564971923828, stop loss 220.25999450683594
13:30:00
opening range high is 228.72999572753906
opening range low is 225.6501007080078
going short, position size 81 shares at planned price 225.69000244140625, stop loss 228.72999572753906
13:30:00
opening range high is 229.9398956298828
opening range low is 227.9199981689453
going long, position size 307 at planned price 228.72000122070312, stop loss 227.9199981689453
13:30:00
opening range high is 232.1898956298828
opening range low is 230.83999633789062
going long, position size 347 at planned price 231.5417938232422, stop loss 230.83999633789062
13:30:00
opening range high is 235.11000061035156
opening range low is 232.88999938964844
going short, position size 169 shares at planned price 233.69000244140625, stop loss 235.11000061035156
closing out position
13:30:00
opening range high is 234.2214050292

Loading BokehJS ...

/Users/melkerliljegren/Documents/Python Repositories/Projects/Intraday_Strategy/.venv/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


Start                     2025-08-08 13:30...
End                       2025-09-19 19:58...
Duration                     42 days 06:28:00
Exposure Time [%]                    42.61033
Equity Final [$]                  23302.32345
Equity Peak [$]                   25317.07263
Commissions [$]                         7.906
Return [%]                           -6.79071
Buy & Hold Return [%]                11.36384
Return (Ann.) [%]                   -41.95221
Volatility (Ann.) [%]                12.09338
CAGR [%]                            -34.24582
Sharpe Ratio                         -3.46902
Sortino Ratio                        -2.83459
Calmar Ratio                         -5.27166
Alpha [%]                             -4.3262
Beta                                 -0.21687
Max. Drawdown [%]                    -7.95807
Avg. Drawdown [%]                    -2.94648
Max. Drawdown Duration       35 days 01:56:00
Avg. Drawdown Duration        8 days 10:52:00
# Trades                          